In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
# Load dataset
data = pd.read_csv("..\data\Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Feature engineering
# Drop irrelevant columns
irrelevant_columns = ['RowNumber', 'CustomerId', 'Surname']
data = data.drop(irrelevant_columns, axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
# Encoding categorical variables
categorical_variables = ['Gender', 'Geography']

In [5]:
# Encoding 'Gender' using LabelEncoder
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [8]:
# Encoding 'Geography' using "One Hot Encoding"
# Because here we have 3 values and might assign like below
# France=0, Spain=1, Germany=2
# This will make the system to think that Germany is of highest priority due to higher number
# So lets use OHE to encode 'Geography'

from sklearn.preprocessing import OneHotEncoder

onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [14]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [12]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [15]:
# To visualise 'Geography' column data in OHE
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [16]:
# Remove 'Geography' from data
data = data.drop(['Geography'], axis = 1)

In [17]:
# Combining OHE columns with original data
data = pd.concat([data, geo_encoded_df], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [21]:
# Save the encoders pickle file

# 1. Saving 'Gender' encoder value
# label_encoder_gender - encoder object that stores the mapping rules (Female = 0 and Male = 1)
with open('../models/encoders/label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

# 2. Saving 'Geography' encoder value
# onehot_encoder_geo - encoder object that stores the mapping rules (France = [1,0,0], Germany=[0,1,0], Spain=[0,0,1])
with open('../models/encoders/onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [22]:
# Divide the dataset into dependent (X) and independent features (y)
X = data.drop('Exited', axis=1) 
y = data['Exited']

In [23]:
# Split the data into training & testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Scale these features using StandardScaler (Mean = 0 & Standard deviation = 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [26]:
X_test

array([[-0.57749609,  0.91324755, -0.6557859 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.29729735,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.52560743, -1.09499335,  0.48508334, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.81311987, -1.09499335,  0.77030065, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.41876609,  0.91324755, -0.94100321, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.24540869,  0.91324755,  0.00972116, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [ ]:
# Save scaler these to pickle files
# "scaler" is an object that remembers training data statistics.
with open('../models/encoders/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Implementation

In [56]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [57]:
print(X_train.shape)
print(f"Total number of columns (input layers) = {X_train.shape[1]}")
print(f"input_shape={(X_train.shape[1],)}")

(8000, 12)
Total number of columns (input layers) = 12
input_shape=(12,)


In [58]:
# Build ANN Model (Sequential Model)
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL1 - Connected with Input layer with 64 neurons
    Dense(32, activation='relu'), # HL2 - No need of input_shape (Connected with HL2 with 64 neurons)
    Dense(1, activation='sigmoid') # Output layer - sigmoid function since it is binary classification
])

In [59]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

# Compiling model using custom optimiser and loss (METHOD 1)
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

# Compiling model (For doing forward & backward propagation) (METHOD 2)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [61]:
# Compiling model (For doing forward & backward propagation)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [63]:
# Setup tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir)

In [64]:
# Setup Early Stopping
# To restrict epochs and stop it once an optimal condition is achieved
# patience = wait atleast till 5 epochs are done. And after that if there is no improvement -- stop
# restore_best_weights = find at which epoch you got best weight and set it as weights once epoch is stopped
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [65]:
# Training the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test),
    epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.3928 - accuracy: 0.8354 - val_loss: 0.3599 - val_accuracy: 0.8565
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3531 - accuracy: 0.8525 - val_loss: 0.3447 - val_accuracy: 0.8610
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3474 - accuracy: 0.8583 - val_loss: 0.3600 - val_accuracy: 0.8500
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3455 - accuracy: 0.8594 - val_loss: 0.3582 - val_accuracy: 0.8460
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3402 - accuracy: 0.8590 - val_loss: 0.3449 - val_accuracy: 0.8590
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3403 - accuracy: 0.8631 - val_loss: 0.3525 - val_accuracy: 0.8575
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3350 - accuracy: 0.8627 - val_loss: 0.3431 - val_accuracy: 0.8585

In [66]:
# Saving the ANN model file
model.save('../models/ann/churn_ann_model.h5')

c:\Dev\projects\bank-churn-prediction-ann\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [67]:
# Load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [68]:
%tensorboard --logdir ../logs/fit

Reusing TensorBoard on port 6006 (pid 17828), started 0:07:44 ago. (Use '!kill 17828' to kill it.)

## PREDICTION WITH ANN MODEL

In [69]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [108]:
# Load ANN trained model
ann_model = load_model(r"..\models\ann\churn_ann_model.h5")

# Load the Label encoder pickle file
with open(r"..\models\encoders\label_encoder_gender.pkl", "rb") as file:
    label_encoder_gender = pickle.load(file)

# Load the OHE pickle file
with open(r"..\models\encoders\onehot_encoder_geo.pkl", "rb") as file:
    onehot_encoder_geo = pickle.load(file)

# Load the scaler pickle file
with open(r"..\models\encoders\scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

In [109]:
# Example Input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [110]:
# Input data to Input DF
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [111]:
# Encoding Gender using DataFrame
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [112]:
# Encoding geography using Dictionary
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Dev\projects\bank-churn-prediction-ann\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [113]:
# Removing 'Geography' and concatenating OHE data 
input_df = input_df.drop(columns="Geography", axis=1)
input_df = pd.concat([input_df, geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [114]:
# Scaling input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [117]:
# Prediction
prediction_churn = ann_model.predict(input_scaled)
prediction_churn

1/1 [==============================] - 0s 36ms/step


array([[0.02140585]], dtype=float32)

In [118]:
prediction_churn_probability = prediction_churn[0][0]
prediction_churn_probability

0.02140585

In [119]:
if prediction_churn_probability > 0.5:
    print('The customer is likely to churn')
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
